We would like to compare and contrast case clustering based on the opinion text (natural language processing) vs. based on the citation structure (network community detection). 

Commmunity detection on the network
- modularity
- walktrap
- SBM (todo)

Clustering on the opinion texts
- compute TD-IDF vectors of opinions
    - KNN on tdidf vectors
    - Gaussian mixture models (TODO)
    - spectral clustering on similarity matrix (TODO)
- topic modeling (TODO)
    - LDA
    - nonegative matrix factorization
    
Relational topic models (see blei paper) (TODO)

### TODO
- match clusters from different algos
- find representatives for clusters
    - top td-idf words
    - 'most central case' in community (is this a thing?)
- more CD algos
    - fix number of communities same as number of NLP clusters 
- more NLP based clustering algos

# Notes

borrowing some code from http://brandonrose.org/clustering

In [3]:
# modify these for your own computer
#repo_directory = '/Users/iaincarmichael/Dropbox/Research/law/law-net/'
repo_directory = '/Users/Michael/Documents/GitHub/law-net/'

#data_dir = '/Users/iaincarmichael/Documents/courtlistener/data/'
data_dir = '/Users/Michael/Desktop/network_data/'

import numpy as np
import sys
import matplotlib.pyplot as plt



# graph package
import igraph as ig


# stat
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans


# our code
sys.path.append(repo_directory + 'code/')


sys.path.append(repo_directory + 'vertex_metrics_experiment/code/')
from bag_of_words import load_tf_idf


# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'
nlp_dir = subnet_dir + 'nlp/'


# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# load the graph
G = ig.Graph.Read_GraphML(subnet_dir + network_name +'_network.graphml')

# largest connected component

restrict our attention to the largest connected componenet on the network. also we are missing some text files from 2016 so lets ignore 2016.

In [5]:
# limit ourselves to cases upto and including 2015 since we are missing some textfiles from 2016
G = G.subgraph(G.vs.select(year_le=2015))

# make graph undirected
Gud = G.copy()
Gud = Gud.as_undirected()

# get largest connected componenet
components = Gud.clusters(mode='STRONG')
g = components.subgraphs()[np.argmax(components.sizes())]

# CL ids of cases in largest connected component
CLids = g.vs['name']

# graph clustering

Do community detection on network

## modularity

In [6]:
%%time 

# modularity clustering
cd_modularity = g.community_fastgreedy() # .as_clustering().membership

mod_clust = cd_modularity.as_clustering()

mod_clust.summary()

Wall time: 1min 30s


In [7]:
graph_clusters = pd.Series(mod_clust.membership, index=g.vs['name'])

## walktrap

In [6]:
# %time cd_walktrap = g.community_walktrap()

# wt_clust = cd_walktrap.as_clustering()

# wt_clust.summary()

# NLP clustering

## load td-idf vectors

In [8]:
tfidf_matrix, op_id_to_bow_id = load_tf_idf(nlp_dir)

## K means clustering on td-idf

In [9]:
%%time

# set number of clusters
num_clusters = 30

# run kmeans
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

nlp_clusters = km.labels_.tolist()

Wall time: 58min 42s


# Compare NLP vs graph clustering

In [13]:
clusters = pd.DataFrame(index=normalized_text_dict.keys(), columns=['nlp', 'graph'])

# add in NLP clusters
clusters['nlp'] = nlp_clusters


# add in communities 
clusters['graph'] = graph_clusters

# consider nodes not considered in CD to be their own cluster
# i.e. nodes outside the largest connected component
clusters['graph'].fillna(max(graph_clusters) + 1, inplace=True)

# make formatting
clusters['graph'] = clusters['graph'].astype(np.int)

NameError: name 'normalized_text_dict' is not defined

In [10]:
clusters

NameError: name 'clusters' is not defined

In [115]:
# TODO: match clusters